# Workbook for optimizing foci segmentation thresholds

This workbook provides step-by-step instructions for determining segmentation thresholds for identifying foci within cells. __Do not edit this workbook file directly!__ Make a copy for each optimization that you perform and edit that version. Save this template in an unedited form for future experiments.

This workbook uses a built-in "optimization mode" that only pulls the first three images out of multi-image .czi files to dramatically speed up processing. This will allow you to tinker with thresholds and find the ideal cutoffs.

I found that the most effective way to define these cutoffs was to use a positive control image - one where there were many foci - and a negative control image with few to no foci to set the thresholds. This allows you to identify cutoffs that simultaneously maximize sensitivity while minimizing false positives.

__Before running this notebook, you must have completed the setup notebook in the install folder.__

In [2]:
# import required code
import sys
sys.path.append('/Users/nweir/Dropbox/code/csth-imaging')
from csth_analysis import find_cells, segment_cells, foci
import os
from skimage import io
import pickle
# uncomment the following lines if you have jupyternotify installed and want to use it
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

/usr/local/lib/python3.6/site-packages/tifffile.py:313: UserWarning: ImportError: No module named '_tifffile'. Loading of some compressed images will be very slow. Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "ImportError: No module named '_tifffile'. "
/usr/local/lib/python3.6/site-packages/czifile.py:130: UserWarning: failed to import the optional _czifile C extension module.
Decoding of JXR and JPEG encoded images will not be available.
Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  "failed to import the optional _czifile C extension module.\n"
/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


<IPython.core.display.Javascript object>

The cell below defines the file paths for the files you will need during this process. Change each variable as needed.

In [4]:
dATG7_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dAtg7_3hrTor_9Pos_1_AiryscanProcessing.czi'
dATG7_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dAtg7_3hrTor_Empty_AiryscanProcessing.czi'
dFIP200_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dFIP200_3hrTor_7Pos_AiryscanProcessing.czi'
dFIP200_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dFIP200_3hrTor_Empty_AiryscanProcessing.czi'
WT_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_WT_3hrTor_6Pos_AiryscanProcessing.czi'
WT_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_WT_3hrTor_Empty_AiryscanProcessing.czi'
dTMEM_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dTMEM_3hrTor_5Pos_AiryscanProcessing.czi'
dTMEM_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dTMEM_3hrTor_Empty_AiryscanProcessing.czi'
dVPS37A_czi = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dVPS37A_3hrTor_10Pos_1_AiryscanProcessing.czi'
dVPS37A_empty = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/Lysoview-LC3_HEK_dVPS37A_3hrTor_Empty_AiryscanProcessing.czi'
svm_pkl = '/Users/nweir/Dropbox/code/csth-imaging/trained_svm.pkl' # change /path/to/ so it points to the csth-imaging folder

Now that you have defined the important variables, you will perform pre-processing. This takes the following steps:
1. Load images
2. Identify regions of the field containing cells
3. Identify nuclei
4. Segment cells  
    _Note:_ By default this segments cells using the 488 wavelength. Change 488 to your desired wavelength in the splitter.segment_cells() command to change wavelengths.
5. Initialize foci detection object.

The steps are indicated in the code blocks.

In [8]:
%%notify
## NOTE: RUNNING THIS CELL WILL TAKE A WHILE! DON'T BE CONCERNED! ##

# 1. load images and the trained SVM focus classifier, and 2.
dVPS37A_finder = find_cells.MultiFinder(filename=dVPS37A_czi,
                                         bg_filename=dVPS37A_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dATG7_finder = find_cells.MultiFinder(filename=dATG7_czi,
                                         bg_filename=dATG7_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dTMEM_finder = find_cells.MultiFinder(filename=dTMEM_czi,
                                         bg_filename=dTMEM_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dFIP200_finder = find_cells.MultiFinder(filename=dFIP200_czi,
                                         bg_filename=dFIP200_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
WT_finder = find_cells.MultiFinder(filename=WT_czi,
                                         bg_filename=WT_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
print('MultiFinder created.')

# 2. find cells
WT_splitter = segment_cells.CellSplitter(WT_finder)
dVPS37A_splitter = segment_cells.CellSplitter(dVPS37A_finder)
dATG7_splitter = segment_cells.CellSplitter(dATG7_finder)
dTMEM_splitter = segment_cells.CellSplitter(dTMEM_finder)
dFIP200_splitter = segment_cells.CellSplitter(dFIP200_finder)

# 3. identify nuclei
WT_splitter.segment_nuclei(verbose=True)
dVPS37A_splitter.segment_nuclei(verbose=True)
dATG7_splitter.segment_nuclei(verbose=True)
dTMEM_splitter.segment_nuclei(verbose=True)
dFIP200_splitter.segment_nuclei(verbose=True)

# 4. Segment cells.
WT_splitter.segment_cells(561, verbose=True)
dVPS37A_splitter.segment_cells(561, verbose=True)
dATG7_splitter.segment_cells(561, verbose=True)
dTMEM_splitter.segment_cells(561, verbose=True)
dFIP200_splitter.segment_cells(561, verbose=True)

# 5. Initialize foci detection objects.
dVPS37A_foci = foci.Foci(dVPS37A_splitter, verbose=True)
dATG7_foci = foci.Foci(dATG7_splitter, verbose=True)
dFIP200_foci = foci.Foci(dFIP200_splitter, verbose=True)
dTMEM_foci = foci.Foci(dTMEM_splitter, verbose=True)
WT_foci = foci.Foci(WT_splitter, verbose=True)

Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
MultiFinder created.
generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1

eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
correcting slice labels...
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
appending outputs...
mask #3 complete.

generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling cont

creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #3 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 3156
slice-by-slice cutoffs:
[   71.     50.5    66.     78.5    83.     93.5    98.    120.5   127.
   134.    154.    173.    238.5   278.5   334.5   373.    414.5   516.5
   810.   1438.5  2387.5  3467.   4421.   5056.5  6008.5  6770.5  7165.5
  7494.   7632.   7564.   7490.5  7533.   7542.5  7765.5  7891.5  7814.
  7580.   7210.5  7065.   6909.   6819.   6506.5  6536.   6453.   5991.5
  5056.5  3849.   2709.   1940.   1517.5  1251.5]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

removing small perinuclear

raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #2 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 2747
slice-by-slice cutoffs:
[  385.5   458.5   692.5  1090.   1726.   2498.5  3417.   4369.5  5262.
  6011.   6437.   6554.   6725.5  6861.5  6716.   6546.5  6462.5  6431.5
  6399.   6438.   6710.   6868.   6759.5  6670.   6597.5  6631.5  6650.
  6540.   6409.5  6313.   6286.5  5980.   5665.   5679.   5500.   5277.5
  5240.   5311.   5437.5  5517.   5368.5  4969.   4801.5  4760.5  4686.5
  4480.5  4189.   3741.   3212.   2568.5  1977.5]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
s

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell mask means:
[2255.4263718826528, 2309.7546333073146, 2362.7866396386676, 2411.9481980187043, 2455.3261914539944, 2492.6310010474072, 2525.3411034090113, 2554.3114748088847, 2580.6936842339719, 2605.1154020295521]
slopes:
[53.680133878007382, 51.096782355694813, 46.269775907663416, 40.341401514351446, 35.007455977508471, 30.840236880738757, 27.676290412480284, 25.401963610333723]
slope deltas:
[ 2.58335152  4.82700645  5.92837439  5.33394554  4.1672191   3.16394647
  2.2743268 ]
desired erosions: 4
erosion of mask #0 complete.
eroding mask 1...
cell mask means:
[2007.1484607062439, 2074.2901798613116, 2143.2168346299318, 2209.0412718918069, 2265.6470153460696, 2305.8306805476445, 2333.5519898943066, 2350.3314016567069, 2349.5453084120436, 2332.6851288526659]
slopes:
[68.034186961843943, 67.375546015247664, 61.215090358068892, 48.394704327918816, 33.952487274118539, 22.250360554531198, 7.9966592588684762, -8.8231364020205092]
slope deltas:
[  0.65864095   6.16045566  12.82038603  14

watershed segmenting cells from nuclei centers...
segmentation complete for image #3.
removing cells that contact the edge of the image.
removing small foci...
2 raw segmented cells
0 foci volume < 1000 px, removing...

checking that nuclei have already been segmented...
eroding cell edges...
eroding mask 0...
cell mask means:
[2232.8175558370799, 2272.6834965331591, 2309.7041869689547, 2343.0228032474652, 2372.1734595470634, 2397.2522415007966, 2418.4136306860091, 2434.7677518369296, 2446.7923422505219, 2454.7912878194561]
slopes:
[38.443315565937382, 35.169653357153038, 31.234636289054379, 27.114719126665705, 23.120085569472849, 18.757755168066524, 14.189355782256371, 10.011767991263241]
slope deltas:
[ 3.27366221  3.93501707  4.11991716  3.99463356  4.3623304   4.56839939
  4.17758779]
desired erosions: 7
erosion of mask #0 complete.
eroding mask 1...
cell mask means:
[2728.1635293182658, 2766.3840525979676, 2800.2759163983296, 2829.0380568426212, 2853.1828621908126, 2873.2073905084

<IPython.core.display.Javascript object>

Next, you're ready to try segmenting foci and checking the output to see if it looks correct.

A bit of background on how Canny edge detection works so you can logically select values. There are two threshold numbers that go into detecting foci:
- high threshold: the minimum edge intensity to _seed_ a new object. The entire object doesn't have to have an edge this sharp, but if no part of it does, it won't be identified.
- low threshold: the minimum edge intensity to _grow_ an object. If one point already satisfied the high threshold, it can grow even if it has a dimmer edge as long as that dimmer edge is above the low threshold.  

If this doesn't make sense, read about the Canny edge detection algorithm on Wikipedia or elsewhere.

I often start with values of 10,000 and 7,500 for the high and low thresholds respectively, so those are the default values below.

Running the code block below will perform segmentation and save the raw images as well as the segmented foci to the output_dir you select in the first line. You can then open these images in fiji, overlay the segmented foci onto the raw images, and determine if the threshold is too high/too low. The code will also output some descriptive information about the number of dots, their intensities, etc. which may be useful to you in deciding thresholds.

In [4]:
from skimage import io  # for saving images
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/'  # change this to where you'd like the output images to go.
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
# save raw images
for c in dATG7_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dATG7_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dAtg7_raw_' + str(c) + '_' + str(im) + '.tif',
                 dATG7_foci.imgs[c][im, :, :, :])
for c in dVPS37A_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dVPS37A_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dVPS37A_raw_' + str(c) + '_' + str(im) + '.tif',
                 dVPS37A_foci.imgs[c][im, :, :, :])    
for c in dTMEM_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dTMEM_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dTMEM_raw_' + str(c) + '_' + str(im) + '.tif',
                 dTMEM_foci.imgs[c][im, :, :, :])
for c in dFIP200_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, dFIP200_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'dFIP200_raw_' + str(c) + '_' + str(im) + '.tif',
                 dFIP200_foci.imgs[c][im, :, :, :])
for c in WT_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, WT_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'WT_raw_' + str(c) + '_' + str(im) + '.tif',
                 WT_foci.imgs[c][im, :, :, :])

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dAtg7_raw_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dAtg7_raw_488_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dAtg7_raw_488_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dAtg7_raw_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWa

In [9]:
def write_big_pkl(obj, path):
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj)
    with open(path, 'wb') as f_out:
        for idx in range(0, len(bytes_out), max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])
    f_out.close()

write_big_pkl(dATG7_foci, '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/dATG7_foci.pkl')
write_big_pkl(dTMEM_foci, '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/dTMEM_foci.pkl')
write_big_pkl(WT_foci, '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/WT_foci.pkl')
write_big_pkl(dFIP200_foci, '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/dFIP200_foci.pkl')
write_big_pkl(dVPS37A_foci, '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/dVPS37A_foci.pkl')

In [2]:
%%notify
def read_big_pkl(path):
    max_bytes = 2**31 - 1
    bytes_in = bytearray(0)
    input_size = os.path.getsize(path)
    with open(path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    return pickle.loads(bytes_in)

dATG7_foci = read_big_pkl('/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/dATG7_foci.pkl')
dTMEM_foci = read_big_pkl('/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/dTMEM_foci.pkl')
WT_foci=read_big_pkl('/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/WT_foci.pkl')
dFIP200_foci=read_big_pkl('/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/dFIP200_foci.pkl')
dVPS37A_foci=read_big_pkl('/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/dVPS37A_foci.pkl')

<IPython.core.display.Javascript object>

## The following cell is for running segmentation on both channels.

In [4]:
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/'
# edit the (high, low) threshold pairs below to optimize segmentation.
green_thresholds = (8000, 6000) 
red_thresholds = (8000, 6000)
# perform segmentation
dATG7_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds},
                   min_cutoff={488: 3, 561: 2})
for c in dATG7_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dATG7_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dATG7_foci_' + str(c) + '_' + str(im) + '.tif',
                 dATG7_foci.foci[c][im])
dVPS37A_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds},
                   min_cutoff={488: 3, 561: 2})
for c in dVPS37A_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dVPS37A_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dVPS37A_foci_' + str(c) + '_' + str(im) + '.tif',
                 dVPS37A_foci.foci[c][im])
dTMEM_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds},
                   min_cutoff={488: 3, 561: 2})
for c in dTMEM_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dTMEM_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dTMEM_foci_' + str(c) + '_' + str(im) + '.tif',
                 dTMEM_foci.foci[c][im])
dFIP200_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds},
                   min_cutoff={488: 3, 561: 2})
for c in dFIP200_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(dFIP200_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'dFIP200_foci_' + str(c) + '_' + str(im) + '.tif',
                 dFIP200_foci.foci[c][im])
WT_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds},
                   min_cutoff={488: 3, 561: 2})
for c in WT_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(WT_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'WT_foci_' + str(c) + '_' + str(im) + '.tif',
                 WT_foci.foci[c][im])

beginning segmentation.
------------------------------------------------------
segmenting foci from channel 488
------------------------------------------------------
canny threshold for channel 488: (8000, 6000)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: nan
normalizing image...


/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.


/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:183: RuntimeWarning: invalid value encountered in greater
  eroded_mask = eroded_mask & (magnitude > 0)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:192: RuntimeWarning: invalid value encountered in greater_equal
  pts_plus = (isobel >= 0) & (jsobel >= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:193: RuntimeWarning: invalid value encountered in less_equal
  pts_minus = (isobel <= 0) & (jsobel <= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:193: RuntimeWarning: invalid value encountered in greater_equal
  pts_minus = (isobel <= 0) & (jsobel <= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:211: RuntimeWarning: invalid value encountered in greater_equal
  pts_plus = (isobel >= 0) & (jsobel >= 0) & (abs_isobel <= abs_jsobel)
/usr/local/lib/python3.6/sit

generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 0 foci in image


...

beginning segmentation.
------------------------------------------------------
segmenting foci from channel 488
------------------------------------------------------
canny threshold for channel 488: (8000, 6000)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 2918.52418541
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 195 foci in image
cell mean: 978.680227306
cell standard deviation: 1018.63162239
intensity cutoff for removal: 4034.57509447
-----filtering foci-----
focus intensity: 44.5653707815
removing focus
focus intensity: 3664.3

  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[50]
matched to one parent cell.
current ID: 16
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[135]
matched to one parent cell.
current ID: 17
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[88]
matched to one parent cell.
current ID: 21
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[12]
matched to one parent cell.
current ID: 23
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[135]
matched to one parent cell.
current ID: 24
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[89]
matched to one parent cell.
current ID: 26
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[57]
matched to one parent cell.
current ID: 27
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[27]
matched to one parent cell.
current ID: 3

current ID: 101
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[66]
matched to one parent cell.
current ID: 102
  overlapping parent cell(s): 
[1 2]
  # of pixels overlapping with parent(s):
[39 20]
  more than one overlapping cell.
  determining which cell overlaps more.
  parent cell: 1
current ID: 103
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[5]
matched to one parent cell.
current ID: 104
  overlapping parent cell(s): 
[1 3]
  # of pixels overlapping with parent(s):
[11 27]
  more than one overlapping cell.
  determining which cell overlaps more.
  parent cell: 3
current ID: 105
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[85]
matched to one parent cell.
current ID: 107
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[95]
matched to one parent cell.
current ID: 108
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[31]
matched to 

current ID: 179
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[90]
matched to one parent cell.
current ID: 180
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[260]
matched to one parent cell.
current ID: 181
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[117]
matched to one parent cell.
current ID: 183
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[142]
matched to one parent cell.
current ID: 184
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[75]
matched to one parent cell.
current ID: 185
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[50]
matched to one parent cell.
current ID: 186
  overlapping parent cell(s): 
[3]
  # of pixels overlapping with parent(s):
[93]
matched to one parent cell.
current ID: 187
  overlapping parent cell(s): 
[1 2]
  # of pixels overlapping with parent(s):
[113  86]
  more tha

KeyboardInterrupt: 

Edit the thresholds in the cell above and re-run as needed until you reach ideal thresholds.

## The following cell is for running segmentation on just one channel.

In [10]:
%%notify
output_dir = '/Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/'
desired_channel = 561  # change this to 488 for green or 561 for red
# edit the (high, low) threshold pair below to optimize segmentation.
thresholds = (4000, 2000) 
# perform segmentation
dATG7_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,), cutoff_type='absolute',
                  min_cutoff={561: 2000})
for im in range(0, len(dATG7_foci.foci[561])):  # for each image in that channel
    io.imsave(output_dir + 'dATG7_foci_561' + '_' + str(im) + '.tif',
             dATG7_foci.foci[561][im])
dVPS37A_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,),
                  min_cutoff={561: 1.5})
for im in range(0, len(dVPS37A_foci.foci[561])):  # for each image in that channel
    io.imsave(output_dir + 'dVPS37A_foci_561' + '_' + str(im) + '.tif',
             dVPS37A_foci.foci[561][im])

beginning segmentation.
------------------------------------------------------
segmenting foci from channel 561
------------------------------------------------------
canny threshold for channel 561: (4000, 2000)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: nan
normalizing image...


/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.


/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:183: RuntimeWarning: invalid value encountered in greater
  eroded_mask = eroded_mask & (magnitude > 0)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:192: RuntimeWarning: invalid value encountered in greater_equal
  pts_plus = (isobel >= 0) & (jsobel >= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:193: RuntimeWarning: invalid value encountered in less_equal
  pts_minus = (isobel <= 0) & (jsobel <= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:193: RuntimeWarning: invalid value encountered in greater_equal
  pts_minus = (isobel <= 0) & (jsobel <= 0) & (abs_isobel >= abs_jsobel)
/usr/local/lib/python3.6/site-packages/skimage/feature/_canny.py:211: RuntimeWarning: invalid value encountered in greater_equal
  pts_plus = (isobel >= 0) & (jsobel >= 0) & (abs_isobel <= abs_jsobel)
/usr/local/lib/python3.6/sit

generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 0 foci in image


...

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_foci_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_foci_561_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_foci_561_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)



------------------------------------------------------
beginning segmentation.
------------------------------------------------------
segmenting foci from channel 561
------------------------------------------------------
canny threshold for channel 561: (4000, 2000)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 2742.14531035
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...


KeyboardInterrupt: 

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [11]:
for i in range(0, len(dATG7_foci.segmented_cells)):
    io.imsave(output_dir + 'dATG7_cells_' + str(i) + '.tif', dATG7_foci.segmented_cells[i])
    io.imsave(output_dir + 'dATG7_nuclei_' + str(i) + '.tif', dATG7_foci.segmented_nuclei[i])
for i in range(0, len(dVPS37A_foci.segmented_cells)):
    io.imsave(output_dir + 'dVPS37A_cells_' + str(i) + '.tif', dVPS37A_foci.segmented_cells[i])
    io.imsave(output_dir + 'dVPS37A_nuclei_' + str(i) + '.tif', dVPS37A_foci.segmented_nuclei[i])

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_cells_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_nuclei_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_cells_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/optimization/lc3_lysoview/outputs/dATG7_nuclei_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning:

# Troubleshooting cell segmentation in these images

In [5]:
%%notify

dVPS37A_finder = find_cells.MultiFinder(filename=dVPS37A_czi,
                                         bg_filename=dVPS37A_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dATG7_finder = find_cells.MultiFinder(filename=dATG7_czi,
                                         bg_filename=dATG7_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dTMEM_finder = find_cells.MultiFinder(filename=dTMEM_czi,
                                         bg_filename=dTMEM_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
dFIP200_finder = find_cells.MultiFinder(filename=dFIP200_czi,
                                         bg_filename=dFIP200_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
WT_finder = find_cells.MultiFinder(filename=WT_czi,
                                         bg_filename=WT_empty,
                                         oof_svm=svm_pkl,
                                         optim=True)
print('MultiFinder created.')


Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
Optimization mode: only using first three images.
MultiFinder created.


<IPython.core.display.Javascript object>